In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
import math

Loading BokehJS ...

In [2]:
df = yf.Ticker('SPY').history(interval ='1d' , period ='12mo')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2023-03-06 00:00:00-05:00 to 2024-03-06 00:00:00-05:00
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           253 non-null    float64
 1   High           253 non-null    float64
 2   Low            253 non-null    float64
 3   Close          253 non-null    float64
 4   Volume         253 non-null    int64  
 5   Dividends      253 non-null    float64
 6   Stock Splits   253 non-null    float64
 7   Capital Gains  253 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 17.8 KB


In [3]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2023-03-06 00:00:00-05:00,398.985976,401.350070,397.961568,398.414673,72795900,0.0,0.0,0.0
2023-03-07 00:00:00-05:00,398.365386,398.611643,391.677031,392.307434,108310600,0.0,0.0,0.0
2023-03-08 00:00:00-05:00,392.425659,393.725875,390.652589,392.947723,74746600,0.0,0.0,0.0
2023-03-09 00:00:00-05:00,393.755436,395.469406,384.683328,385.697906,111945300,0.0,0.0,0.0
2023-03-10 00:00:00-05:00,385.136441,387.273967,378.566315,380.132507,189253000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-02-29 00:00:00-05:00,508.070007,509.739990,505.350006,508.079987,83924800,0.0,0.0,0.0
2024-03-01 00:00:00-05:00,508.980011,513.289978,508.559998,512.849976,76805900,0.0,0.0,0.0
2024-03-04 00:00:00-05:00,512.030029,514.200012,512.000000,512.299988,49799300,0.0,0.0,0.0


In [4]:
class DCA(Strategy):

    amount_to_invest = 10

    def init(self):
        self.day_of_week = self.I(lambda x: x , self.data.Close.s.index.dayofweek)
        # print(self.day_of_week)            

    def next(self):
        # print(len(self.data.Close))
        if self.day_of_week[-1] == 0: #Assume buying at every Monday's closing price.
            self.buy( size = math.floor(self.amount_to_invest / self.data.Close[-1]))
            #math.floor :Round numbers down to the nearest integer
            #If you enter a float between 0 and 1, backtesting.py will assume that you want to invest that fraction of your available cash. 
            #If you enter an integer >= 1, then it will assume that you want to purchase that number of shares.
            # ex. 10usd/400usd =0.025shares =2500 micro-shares   So, the solution is to convert the price to micro-shares.
            try:
                if self.data.Close[-1]/self.data.Close[-30] < 0.90:  #If the price has decreased by more than 10% in the last month 
                    self.buy( size = math.floor(self.amount_to_invest / self.data.Close[-1]))
            except:
                pass


df= df*10**-6   #turn to micro-shares         
bt = Backtest(df,DCA,trade_on_close = True, cash = 10000 )
stats = bt.run()
bt.plot()
print(stats)

Start                     2023-03-06 00:00...
End                       2024-03-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   97.628458
Equity Final [$]                  10071.90546
Equity Peak [$]                  10077.729234
Return [%]                           0.719055
Buy & Hold Return [%]               27.929426
Return (Ann.) [%]                    0.716202
Volatility (Ann.) [%]                0.335547
Sharpe Ratio                         2.134435
Sortino Ratio                        3.492045
Calmar Ratio                         2.620401
Max. Drawdown [%]                   -0.273318
Avg. Drawdown [%]                   -0.031265
Max. Drawdown Duration      107 days 00:00:00
Avg. Drawdown Duration       10 days 00:00:00
# Trades                                   45
Win Rate [%]                        97.777778
Best Trade [%]                      33.612334
Worst Trade [%]                     -0.999413
Avg. Trade [%]                    

In [5]:
print(stats['_trades'].to_string())

     Size  EntryBar  ExitBar  EntryPrice  ExitPrice       PnL  ReturnPct                 EntryTime                  ExitTime          Duration
0   19519       250      251    0.000512   0.000507 -0.099937  -0.009994 2024-03-04 00:00:00-05:00 2024-03-05 00:00:00-05:00   1 days 00:00:00
1   19763       245      251    0.000506   0.000507  0.023518   0.002352 2024-02-26 00:00:00-05:00 2024-03-05 00:00:00-05:00   8 days 00:00:00
2   19960       236      251    0.000501   0.000507  0.123752   0.012376 2024-02-12 00:00:00-05:00 2024-03-05 00:00:00-05:00  22 days 00:00:00
3   20302       231      251    0.000493   0.000507  0.297018   0.029703 2024-02-05 00:00:00-05:00 2024-03-05 00:00:00-05:00  29 days 00:00:00
4   20355       226      251    0.000491   0.000507  0.323848   0.032385 2024-01-29 00:00:00-05:00 2024-03-05 00:00:00-05:00  36 days 00:00:00
5   20684       221      251    0.000483   0.000507  0.490831   0.049085 2024-01-22 00:00:00-05:00 2024-03-05 00:00:00-05:00  43 days 00:00:00

In [6]:
trades = stats["_trades"]
price_paid = trades["Size"] * trades["EntryPrice"]
total_invested = price_paid.sum()

current_shares = trades["Size"].sum()
current_equity = current_shares * df.Close.iloc[-1]

print("Total investment:",f'{total_invested} usd')
print("Current Shares:",current_shares / (10**6))
print("current Equity:", f'{current_equity} usd')

print("Return:", f'{current_equity*100 / total_invested :.3f} %')


Total investment: 449.99137412109377 usd
Current Shares: 1.029017
current Equity: 524.4792690018616 usd
Return: 116.553 %
